# Tarea 2a: resumen de un texto con archivos pequeños con Titan Text Premier


En este cuaderno, se ingiere una pequeña cadena de texto directamente en la API de Amazon Bedrock (mediante el modelo Titan Text) y se le indica que resuma el texto introducido. Puede aplicar este enfoque para resumir transcripciones de llamadas y de reuniones, libros, artículos, publicaciones de blogs y otro contenido pertinente cuando la longitud del texto introducido se encuentre dentro de los límites de tamaño del contexto del modelo.

## Tarea 2a.1: configuración del entorno

En esta tarea, establecerá el entorno.

In [ ]:
#Create a service client by name using the default session.
import json
import os
import sys
import warnings

import boto3
import botocore

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))

bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))



## Tarea 2a.2: escribir petición con texto que se debe resumir

En esta tarea, se utiliza un pasaje de texto corto con menos tokens que la longitud máxima admitida por el modelo fundacional. Como texto de entrada de muestra para este laboratorio, utilice un párrafo de una [publicación del blog de AWS](https://aws.amazon.com/jp/blogs/machine-learning/announcing-new-tools-for-building-with-generative-ai-on-aws/) en la que se anuncia Amazon Bedrock.

La petición comienza con una instrucción `Please provide a summary of the following text.` (Proporcionar un resumen para el siguiente texto). 

In [ ]:
prompt_data = """

Please provide a summary of the following text:

AWS took all of that feedback from customers, and today we are excited to announce Amazon Bedrock, \
a new service that makes FMs from AI21 Labs, Anthropic, Stability AI, and Amazon accessible via an API. \
Bedrock is the easiest way for customers to build and scale generative AI-based applications using FMs, \
democratizing access for all builders. Bedrock will offer the ability to access a range of powerful FMs \
for text and images—including Amazons Titan FMs, which consist of two new LLMs we’re also announcing \
today—through a scalable, reliable, and secure AWS managed service. With Bedrock’s serverless experience, \
customers can easily find the right model for what they’re trying to get done, get started quickly, privately \
customize FMs with their own data, and easily integrate and deploy them into their applications using the AWS \
tools and capabilities they are familiar with, without having to manage any infrastructure (including integrations \
with Amazon SageMaker ML features like Experiments to test different models and Pipelines to manage their FMs at scale).
"""

## Tarea 2a.3: crear el cuerpo de la solicitud con los parámetros de petición y de inferencia 

En esta tarea, se crea el cuerpo de la solicitud con los parámetros de petición y de inferencia anteriores.

In [ ]:
# request body
body = json.dumps({
    "inputText": prompt_data, 
    "textGenerationConfig":{
        "maxTokenCount":2048,
        "stopSequences":[],
        "temperature":0,
        "topP":0.9
        }
    }) 

## Tarea 2a.4: invocar el modelo fundacional a través de Boto3

En esta tarea, envía una solicitud de API a Amazon Bedrock que especifica los parámetros de solicitud: `modelId`, `accept` y `ContentType`. A partir de la petición proporcionada, el modelo fundacional de Amazon Bedrock resume el texto introducido.

### Completar la generación de la salida

De manera predeterminada, el servicio Amazon Bedrock genera el resumen completo de una petición determinada en una única salida. Esto puede ser lento si la salida del modelo contiene muchos tokens. 

In [ ]:
#model configuration and invoke the model
modelId = 'amazon.titan-text-premier-v1:0' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'
outputText = "\n"

try:

    response = bedrock_client.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())

    outputText = response_body.get('results')[0].get('outputText')

except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

print(outputText)

### Generación de la salida de streaming

A continuación, analizará cómo utilizar la API invoke_model_with_response_stream de Amazon Bedrock para transmitir los resultados del modelo, para que los usuarios puedan consumir las salidas a medida que se generan. En lugar de generar la salida completa de una vez, esta API devuelve una propiedad ResponseStream que envía fragmentos de salida más pequeños del modelo a medida que se generan. Puede mostrar estas salidas de streaming en una vista continua y consumible.

In [ ]:
#invoke model with response stream
modelId = 'amazon.titan-text-premier-v1:0'
response = bedrock_client.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = list(stream)
output

In [ ]:
from IPython.display import display_markdown,Markdown,clear_output

In [ ]:
modelId = 'amazon.titan-text-premier-v1:0'
response = bedrock_client.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = []
i = 1
if stream:
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            chunk_obj = json.loads(chunk.get('bytes').decode())
            text = chunk_obj['outputText']
            clear_output(wait=True)
            output.append(text)
            display_markdown(Markdown(''.join(output)))
            i+=1

Experimentó con el uso del SDK boto3 para obtener acceso a la API de Amazon Bedrock. Este SDK proporciona acceso programático básico a las capacidades de Bedrock. Al aprovechar esta API, pudo implementar dos casos prácticos: 1) generar un resumen de texto completo del contenido de noticias de AWS de una sola vez y 2) transmitir la salida del resumen en fragmentos para su procesamiento progresivo.

### Pruébelo usted mismo
- Cambie las peticiones para que se adapten al caso práctico específico y evalúe la salida de los diferentes modelos.
- Juegue con la longitud del token para comprender la latencia y la capacidad de respuesta del servicio.
- Aplique diferentes principios de ingeniería de peticiones para obtener mejores salidas.

### Limpieza

Ha completado este cuaderno. Para ir a la siguiente parte del laboratorio, complete estos pasos:

- Cierre este archivo de cuaderno y continúe con **Task2b.ipynb**.